In [1]:
import face_recognition
import os, sys
import cv2
import numpy as np
import math

In [ ]:
import cv2

img = cv2.imread('image.jpg')

cv2.imshow('Image', img)
cv2.waitKey(0)
#oli

-1

In [6]:
import cv2
import face_recognition

# Imagen a comparar
image = cv2.imread("faces/Felipe_Cespedes_Urzua.jpg")
face_loc = face_recognition.face_locations(image)[0]
#print("face_loc:", face_loc)
face_image_encodings = face_recognition.face_encodings(image, known_face_locations=[face_loc])[0]
print("face_image_encodings:", face_image_encodings)
'''
cv2.rectangle(image, (face_loc[3], face_loc[0]), (face_loc[1], face_loc[2]), (0, 255, 0))
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()'''

######################################################################################
# Video Streaming

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

while True:
     ret, frame = cap.read()
     if ret == False: break
     frame = cv2.flip(frame, 1)

     face_locations = face_recognition.face_locations(frame, model="cnn")
     if face_locations != []:
          for face_location in face_locations:
               face_frame_encodings = face_recognition.face_encodings(frame, known_face_locations=[face_location])[0]
               result = face_recognition.compare_faces([face_image_encodings], face_frame_encodings)
               #print("Result:", result)

               if result[0] == True:
                    text = "Gaby"
                    color = (125, 220, 0)
               else:
                    text = "Desconocido"
                    color = (50, 50, 255)

               cv2.rectangle(frame, (face_location[3], face_location[2]), (face_location[1], face_location[2] + 30), color, -1)
               cv2.rectangle(frame, (face_location[3], face_location[0]), (face_location[1], face_location[2]), color, 2)
               cv2.putText(frame, text, (face_location[3], face_location[2] + 20), 2, 0.7, (255, 255, 255), 1)


     cv2.imshow("Frame", frame)
     k = cv2.waitKey(1)
     if k == 27 & 0xFF:
          break

cap.release()
cv2.destroyAllWindows()

face_image_encodings: [-0.09845468  0.01625447  0.02332346  0.01001488 -0.04612962  0.01842581
 -0.06964554 -0.10699138  0.15465328 -0.03715706  0.23073959  0.02320622
 -0.25251725 -0.02446893  0.02454933  0.12453987 -0.09646164 -0.1331812
 -0.05011172  0.00309733  0.04137759  0.07872947  0.02073788  0.08951613
 -0.13320771 -0.31909928 -0.0266559  -0.17681997  0.01375719 -0.10046207
 -0.07718664  0.04137538 -0.09598592  0.01252296  0.07819108  0.03136981
 -0.04590725 -0.07848373  0.24784338  0.09417851 -0.17443231  0.0514416
 -0.01895836  0.20548728  0.11226749  0.04793255  0.03565768 -0.10268201
  0.13712631 -0.30131102  0.07763261  0.13341351  0.11487656  0.02902473
  0.12729715 -0.23279262 -0.00845373  0.12459449 -0.27904469  0.1636288
 -0.0019316  -0.02995491 -0.00367536 -0.07595484  0.21626322  0.14322986
 -0.1489014  -0.02898763  0.11412504 -0.08915456 -0.08351523  0.07452131
 -0.1158954  -0.26842988 -0.22065689  0.07172944  0.35144648  0.186877
 -0.24387215  0.01155113  0.006395

KeyboardInterrupt: 

In [13]:
def face_confidence(face_distance, face_match_threshold=0.7):
    range = (1.0 - face_match_threshold)
    linear_val = (1.0 - face_distance) / (range * 2.0)

    if face_distance > face_match_threshold:
        return str(round(linear_val * 100, 2)) + "%"
    else:
        value = (linear_val +((1.0 - linear_val) * math.pow((linear_val - 0.5) * 2, 0.2))) * 100
        return str(round(value, 2)) + "%"
 

class FaceRecognition:
    face_locations = []
    face_encodings = []
    face_names = []
    known_face_encodings = []
    known_face_names = []
    process_current_frame = True

    def __init__(self):
        self.encode_faces()
        # encode faces

    def encode_faces(self):
        for image in os.listdir("faces"):
            face_image = face_recognition.load_image_file("faces/" + image)
            face_locations = face_recognition.face_locations(face_image)
            if len(face_locations) > 0:
                face_encoding = face_recognition.face_encodings(face_image, face_locations)[0]
            else:
                print(f"No face detected in image: {image}")
                continue

            self.known_face_encodings.append(face_encoding)
            self.known_face_names.append(image)

            print(self.known_face_names)

    def run_recognition(self):
        video_capture = cv2.VideoCapture(0)

        if not video_capture.isOpened():
            sys.exit("Could not open video device")

        while True:
            ret, frame = video_capture.read()

            if not ret:
                break

            if self.process_current_frame:
                small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
                rgb_small_frame = small_frame[:, :, ::-1]

                self.face_locations = face_recognition.face_locations(rgb_small_frame)
                self.face_encodings = face_recognition.face_encodings(rgb_small_frame, self.face_locations)

                self.face_names = []
                for face_encoding in self.face_encodings:
                    matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
                    name = "Unknown"
                    confidence = "Unknown"

                    face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
                    best_match_index = np.argmin(face_distances)
                    if matches[best_match_index]:
                        name = self.known_face_names[best_match_index]
                        confidence = face_confidence(face_distances[best_match_index])

                    self.face_names.append(f"{name} ({confidence})")

            self.process_current_frame = not self.process_current_frame

            for (top, right, bottom, left), name in zip(self.face_locations, self.face_names):
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), -1)
                cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 1)

            cv2.imshow("Face Recognition", frame)

            if cv2.waitKey(1) == ord("q"):
                break

        video_capture.release()
        cv2.destroyAllWindows()




if __name__ == "__main__":
    fr = FaceRecognition()
    fr.run_recognition()


['Felipe_Cespedes_Urzua.jpg']
['Felipe_Cespedes_Urzua.jpg', 'Mateo_Moreira.jpeg']


TypeError: compute_face_descriptor(): incompatible function arguments. The following argument types are supported:
    1. (self: _dlib_pybind11.face_recognition_model_v1, img: numpy.ndarray[(rows,cols,3),numpy.uint8], face: _dlib_pybind11.full_object_detection, num_jitters: int = 0, padding: float = 0.25) -> _dlib_pybind11.vector
    2. (self: _dlib_pybind11.face_recognition_model_v1, img: numpy.ndarray[(rows,cols,3),numpy.uint8], num_jitters: int = 0) -> _dlib_pybind11.vector
    3. (self: _dlib_pybind11.face_recognition_model_v1, img: numpy.ndarray[(rows,cols,3),numpy.uint8], faces: _dlib_pybind11.full_object_detections, num_jitters: int = 0, padding: float = 0.25) -> _dlib_pybind11.vectors
    4. (self: _dlib_pybind11.face_recognition_model_v1, batch_img: list[numpy.ndarray[(rows,cols,3),numpy.uint8]], batch_faces: list[_dlib_pybind11.full_object_detections], num_jitters: int = 0, padding: float = 0.25) -> _dlib_pybind11.vectorss
    5. (self: _dlib_pybind11.face_recognition_model_v

In [5]:
cap = cv2.VideoCapture(0)  # or your video source
if not cap.isOpened():
    print("Error: Could not open video capture")
    exit()

ret, frame = cap.read()
if not ret or frame is None:
    print("Error: Could not read frame")
    exit()

# Only display if frame has valid dimensions
if frame.shape[1] > 0 and frame.shape[0] > 0:
    cv2.imshow("Face Recognition", frame)

In [ ]:
backends = ["opencv", ]